# Segment Anything Model for Geospatial Data

[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/segment-geospatial/blob/main/docs/examples/satellite.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/satellite.ipynb)

This notebook shows how to use segment satellite imagery using the Segment Anything Model (SAM) with a few lines of code.

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator.

## Install dependencies

Uncomment and run the following cell to install the required dependencies.


In [29]:
pip install segment-geospatial

## Import libraries

In [30]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff, get_basemaps

## Create an interactive map

In [31]:
m = leafmap.Map(center=[29.676840, -95.369222], zoom=19)
m.add_basemap("SATELLITE")
m

Map(center=[29.67684, -95.369222], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [32]:
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [-95.3704, 29.6762, -95.368, 29.6775]

## Download map tiles

Download maps tiles and mosaic them into a single GeoTIFF file

In [33]:
image = "satellite.tif"

Besides the `satellite` basemap, you can use any of the following basemaps returned by the `get_basemaps()` function:

In [ ]:
# get_basemaps().keys()

Specify the basemap as the source.

In [34]:
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source="Satellite", overwrite=True)

Downloaded image 001/578
Downloaded image 002/578
Downloaded image 003/578
Downloaded image 004/578
Downloaded image 005/578
Downloaded image 006/578
Downloaded image 007/578
Downloaded image 008/578
Downloaded image 009/578
Downloaded image 010/578
Downloaded image 011/578
Downloaded image 012/578
Downloaded image 013/578
Downloaded image 014/578
Downloaded image 015/578
Downloaded image 016/578
Downloaded image 017/578
Downloaded image 018/578
Downloaded image 019/578
Downloaded image 020/578
Downloaded image 021/578
Downloaded image 022/578
Downloaded image 023/578
Downloaded image 024/578
Downloaded image 025/578
Downloaded image 026/578
Downloaded image 027/578
Downloaded image 028/578
Downloaded image 029/578
Downloaded image 030/578
Downloaded image 031/578
Downloaded image 032/578
Downloaded image 033/578
Downloaded image 034/578
Downloaded image 035/578
Downloaded image 036/578
Downloaded image 037/578
Downloaded image 038/578
Downloaded image 039/578
Downloaded image 040/578


You can also use your own image. Uncomment and run the following cell to use your own image.

In [ ]:
# image = '/path/to/your/own/image.tif'

Display the downloaded image on the map.

In [35]:
m.layers[-1].visible = False  # turn off the basemap
m.add_raster(image, layer_name="Image")
m

Map(bottom=15129100.0, center=[17.411099999999998, 82.53455], controls=(ZoomControl(options=['position', 'zoom…

![](https://i.imgur.com/KAm84IY.png)

## Initialize SAM class

In [36]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    sam_kwargs=None,
)

## Segment the image

Set `batch=True` to segment the image in batches. This is useful for large images that cannot fit in memory.

In [ ]:
mask = "segment.tif"
sam.generate(
    image, mask, batch=True, foreground=True, erosion_kernel=(3, 3), mask_multiplier=255
)

 29%|██▊       | 39/136 [04:17<10:28,  6.48s/it]

## Polygonize the raster data

Save the segmentation results as a GeoPackage file.

In [ ]:
vector = "segment.gpkg"
sam.tiff_to_gpkg(mask, vector, simplify_tolerance=None)

You can also save the segmentation results as any vector data format supported by GeoPandas.

In [ ]:
shapefile = "segment.shp"
sam.tiff_to_vector(mask, shapefile)

## Visualize the results

In [ ]:
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
m.add_vector(vector, layer_name="Vector", style=style)
m

![](https://i.imgur.com/Ysq3u7E.png)